<h1 align="center"><a href="https://github.com/sborquez/her2bdl"> Her2BDL</a> - Her2 Bayesian Deep Learning</h1>

<center>
<br>
<a href="https://github.com/sborquez/her2bdl"><img src="https://raw.githubusercontent.com/yast-ia/YastAI/master/notebooks/images/header.png" width="35%"/></a>
<h2 align="center">Model Evaluation</h2>
</center>

<center>
<i> Notebook created by Sebastián Bórquez G. - <a href="mailto://sebstian.borquez.g@gmail.com">sebastian.borquez.g@gmail.com</a> - YastAI - August 2020.</i>
</center>


# Setup Notebook

The first step is to sync this notebook with Google Drive, and change directory to gerumo repository.

## (Option A) Colab Setup

Connecto to your `Google Drive` and install some dependencies.

In [3]:
!pip list

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/<Path to Project>"
!ls

## (Option B) Local Setup

Change working directory to project root.

In [5]:
%cd ..

D:\sebas\Google Drive\yastai\projects\YastAI


## Import Modules

In [7]:
# AI packege
from AI import *

# Adhoc modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.core.display import display, HTML

# Model Evaluation

Load and predict with your trained models. Evaluate and plot some metrics.

In [ ]:
# You Know what to do 